In [1]:
# Databricks notebook source
# MAGIC %md
# MAGIC # PHASE 1 - STEP 2: SYNTHETIC DATA GENERATOR (WITH AUDIT COLUMNS)
# MAGIC 
# MAGIC **Version:** 2.0 (Phase 1 Compatible)
# MAGIC 
# MAGIC **Changes from v1.0:**
# MAGIC - ✅ Added support for audit columns (created_timestamp, updated_timestamp, is_deleted)
# MAGIC - ✅ New records get proper timestamps (not NULL)
# MAGIC - ✅ Maintains backward compatibility
# MAGIC - ✅ Auto-detects schema changes
# MAGIC 
# MAGIC **Purpose:** Generate synthetic person records for testing incremental load pattern
# MAGIC 
# MAGIC **Compatible with:** Phase 1 ETL v4.0 (incremental load)

# COMMAND ----------

import pyspark.sql.functions as F
from pyspark.sql.types import *
import time
import builtins
from datetime import datetime

print("=" * 80)
print("SYNTHETIC DATA GENERATOR v2.0 - PHASE 1 COMPATIBLE")
print("=" * 80)
print(f"Spark Version: {spark.version}")
print(f"Execution Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

# COMMAND ----------

# MAGIC %md
# MAGIC ## CONFIGURATION

# COMMAND ----------

# ==============================================================================
# CONFIGURATION
# ==============================================================================
ROWS_TO_GENERATE = 1000000  # Adjust as needed (tested up to 1M)
TARGET_TABLE = "Lake24.dbo.person"

# Audit column behavior
ENABLE_AUDIT_COLUMNS = True  # Set to False to generate NULL (for testing)

print(f"📋 CONFIGURATION:")
print(f"   Target Table: {TARGET_TABLE}")
print(f"   Rows to Generate: {ROWS_TO_GENERATE:,}")
print(f"   Audit Columns: {'Enabled ✅' if ENABLE_AUDIT_COLUMNS else 'Disabled (NULL)'}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## STEP 1: LIVE METADATA ANALYSIS

# COMMAND ----------

print(f"\n🎬 Starting generation of {ROWS_TO_GENERATE:,} records...")

# ==============================================================================
# LIVE METADATA ANALYSIS (DYNAMIC)
# ==============================================================================
print(f"\n🔍 TABLE ANALYSIS:")
print("-" * 80)

# Load the target table once to get both schema and current stats
target_table_df = spark.table(TARGET_TABLE)
actual_schema = target_table_df.schema

# Get the LIVE count and MAX ID
stats_df = target_table_df.select(
    F.count("*").alias("current_count"), 
    F.max("person_id").alias("max_id")
).collect()[0]

initial_row_count = stats_df["current_count"]
max_id_val = stats_df["max_id"]
start_id = (max_id_val or 0) + 1

print(f"   Existing Records: {initial_row_count:,}")
print(f"   Current Max ID:   {max_id_val:,}")
print(f"   Next ID:          {start_id:,}")

# Check for audit columns
schema_columns = [f.name for f in actual_schema.fields]
audit_columns_present = {
    'created_timestamp': 'created_timestamp' in schema_columns,
    'updated_timestamp': 'updated_timestamp' in schema_columns,
    'is_deleted': 'is_deleted' in schema_columns
}

print(f"\n📊 SCHEMA ANALYSIS:")
print(f"   Total Columns:      {len(schema_columns)}")
print(f"   created_timestamp:  {'✅ Present' if audit_columns_present['created_timestamp'] else '❌ Missing'}")
print(f"   updated_timestamp:  {'✅ Present' if audit_columns_present['updated_timestamp'] else '❌ Missing'}")
print(f"   is_deleted:         {'✅ Present' if audit_columns_present['is_deleted'] else '❌ Missing'}")

if all(audit_columns_present.values()):
    print(f"   Status: ✅ Phase 1 schema detected (audit columns present)")
else:
    print(f"   Status: ⚠️  Pre-Phase 1 schema (audit columns missing)")
    print(f"   Note: Audit columns will be set to NULL")

print("-" * 80)

# COMMAND ----------

# MAGIC %md
# MAGIC ## STEP 2: VECTORIZED SYNTHETIC GENERATION

# COMMAND ----------

print(f"\n⚡ GENERATING {ROWS_TO_GENERATE:,} SYNTHETIC RECORDS:")
print("-" * 80)
start_time = time.time()

# ==============================================================================
# BASE SYNTHETIC GENERATION (Original 18 columns)
# ==============================================================================

df_synth = spark.range(0, ROWS_TO_GENERATE) \
    .withColumn("person_id", (F.lit(start_id) + F.col("id")).cast(IntegerType())) \
    .withColumn("gender_rand", F.rand()) \
    .withColumn("gender_concept_id", 
        F.when(F.col("gender_rand") > 0.52, 8532)
         .otherwise(8507)
         .cast(IntegerType())) \
    .withColumn("year_of_birth", (F.rand() * 70 + 1950).cast(IntegerType())) \
    .withColumn("month_of_birth", (F.rand() * 11 + 1).cast(IntegerType())) \
    .withColumn("day_of_birth", (F.rand() * 27 + 1).cast(IntegerType())) \
    .withColumn("race_rand", F.rand()) \
    .withColumn("race_concept_id", 
        F.when(F.col("race_rand") < 0.60, 8527)
         .when(F.col("race_rand") < 0.74, 8515)
         .otherwise(0)
         .cast(IntegerType())) \
    .withColumn("ethnicity_concept_id", F.lit(38003564).cast(IntegerType())) \
    .withColumn("person_source_value", 
        F.concat(F.lit("SYNTH_"), F.col("person_id").cast("string"))) \
    .withColumn("gender_source_value", 
        F.when(F.col("gender_concept_id") == 8507, "M")
         .otherwise("F")) \
    .withColumn("gender_source_concept_id", 
        F.col("gender_concept_id").cast(IntegerType())) \
    .withColumn("birth_datetime", 
        F.to_date(F.concat_ws("-", 
            F.col("year_of_birth"), 
            F.col("month_of_birth"), 
            F.col("day_of_birth"))))

print(f"   ✅ Generated base 18 columns")

# ==============================================================================
# ADD AUDIT COLUMNS (Phase 1 Enhancement)
# ==============================================================================

if ENABLE_AUDIT_COLUMNS and all(audit_columns_present.values()):
    # Audit columns are present in schema AND enabled in config
    current_ts = datetime.now()
    
    df_synth = df_synth \
        .withColumn("created_timestamp", F.lit(current_ts).cast(TimestampType())) \
        .withColumn("updated_timestamp", F.lit(current_ts).cast(TimestampType())) \
        .withColumn("is_deleted", F.lit(False).cast(BooleanType()))
    
    print(f"   ✅ Added audit columns with timestamps")
    print(f"      - created_timestamp: {current_ts}")
    print(f"      - updated_timestamp: {current_ts}")
    print(f"      - is_deleted: False")
else:
    # Either disabled or columns don't exist - will be handled by schema alignment
    print(f"   ⚠️  Audit columns will be auto-filled (NULL or default)")

# ==============================================================================
# SCHEMA ALIGNMENT (Auto-adapts to target table schema)
# ==============================================================================

print(f"\n🔧 SCHEMA ALIGNMENT:")

final_columns = []
aligned_count = 0
auto_filled_count = 0

for field in actual_schema.fields:
    if field.name in df_synth.columns:
        # Column exists in generated data - cast to target type
        final_columns.append(F.col(field.name).cast(field.dataType))
        aligned_count += 1
    else:
        # Column doesn't exist - auto-fill with NULL
        final_columns.append(F.lit(None).cast(field.dataType).alias(field.name))
        auto_filled_count += 1
        print(f"   ⚠️  Auto-filled: {field.name} → NULL")

final_df = df_synth.select(final_columns)

print(f"   ✅ Aligned: {aligned_count} columns")
if auto_filled_count > 0:
    print(f"   ⚠️  Auto-filled: {auto_filled_count} columns with NULL")
else:
    print(f"   ✅ Perfect match: All target columns populated")

generation_time = time.time() - start_time
print(f"\n   Generation Time: {generation_time:.2f} seconds")
print("-" * 80)

# COMMAND ----------

# MAGIC %md
# MAGIC ## STEP 3: QUALITY CHECKS (Pre-Insert Validation)

# COMMAND ----------

print(f"\n🔍 PRE-INSERT QUALITY CHECKS:")
print("-" * 80)

try:
    # 1. Record count check
    generated_count = final_df.count()
    print(f"   ✅ Record count: {generated_count:,} (expected: {ROWS_TO_GENERATE:,})")
    
    if generated_count != ROWS_TO_GENERATE:
        print(f"   ⚠️  WARNING: Count mismatch!")
    
    # 2. Sample data check
    print(f"\n   📊 Sample data (first 3 records):")
    sample_cols = ["person_id", "gender_concept_id", "year_of_birth"]
    if ENABLE_AUDIT_COLUMNS and all(audit_columns_present.values()):
        sample_cols.extend(["created_timestamp", "updated_timestamp", "is_deleted"])
    
    final_df.select(sample_cols).show(3, truncate=False)
    
    # 3. Null checks on key columns
    null_person_id = final_df.filter(F.col("person_id").isNull()).count()
    null_gender = final_df.filter(F.col("gender_concept_id").isNull()).count()
    
    print(f"\n   📊 Null checks:")
    print(f"      person_id nulls: {null_person_id} {'✅' if null_person_id == 0 else '❌'}")
    print(f"      gender_concept_id nulls: {null_gender} {'✅' if null_gender == 0 else '❌'}")
    
    # 4. Audit column validation
    if ENABLE_AUDIT_COLUMNS and all(audit_columns_present.values()):
        null_created = final_df.filter(F.col("created_timestamp").isNull()).count()
        null_updated = final_df.filter(F.col("updated_timestamp").isNull()).count()
        deleted_true = final_df.filter(F.col("is_deleted") == True).count()
        
        print(f"      created_timestamp nulls: {null_created} {'✅' if null_created == 0 else '⚠️'}")
        print(f"      updated_timestamp nulls: {null_updated} {'✅' if null_updated == 0 else '⚠️'}")
        print(f"      is_deleted = True: {deleted_true} {'✅' if deleted_true == 0 else '⚠️'}")
    
    print(f"\n   ✅ All quality checks passed")
    
except Exception as e:
    print(f"   ❌ Quality check failed: {str(e)}")
    raise

print("-" * 80)

# COMMAND ----------

# MAGIC %md
# MAGIC ## STEP 4: APPEND TO DELTA TABLE

# COMMAND ----------

print(f"\n💾 WRITING TO TABLE:")
print("-" * 80)

write_start = time.time()

try:
    final_df.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(TARGET_TABLE)
    
    write_time = time.time() - write_start
    print(f"   ✅ SUCCESS: Write completed in {write_time:.2f} seconds")
    
except Exception as e:
    print(f"   ❌ ERROR: {str(e)}")
    print(f"   Rolling back...")
    raise

print("-" * 80)

# COMMAND ----------

# MAGIC %md
# MAGIC ## STEP 5: POST-INSERT VERIFICATION

# COMMAND ----------

print(f"\n🔍 POST-INSERT VERIFICATION:")
print("-" * 80)

# Get final count
final_count = spark.table(TARGET_TABLE).count()
expected_count = initial_row_count + ROWS_TO_GENERATE

print(f"   Initial rows:  {initial_row_count:,}")
print(f"   Added rows:    {ROWS_TO_GENERATE:,}")
print(f"   Expected:      {expected_count:,}")
print(f"   Actual:        {final_count:,}")

if final_count == expected_count:
    print(f"   ✅ Count verification PASSED")
else:
    print(f"   ❌ Count verification FAILED")
    print(f"   Difference: {final_count - expected_count:,}")

# Verify last inserted records
print(f"\n   📊 Verifying last inserted records:")
last_records = spark.table(TARGET_TABLE) \
    .filter(F.col("person_id") >= start_id) \
    .orderBy(F.col("person_id").desc()) \
    .limit(3)

verify_cols = ["person_id", "gender_concept_id"]
if ENABLE_AUDIT_COLUMNS and all(audit_columns_present.values()):
    verify_cols.extend(["created_timestamp", "is_deleted"])

last_records.select(verify_cols).show(3, truncate=False)

print("-" * 80)

# COMMAND ----------

# MAGIC %md
# MAGIC ## FINAL SUMMARY

# COMMAND ----------

total_time = time.time() - start_time
throughput = ROWS_TO_GENERATE / total_time if total_time > 0 else 0

print("\n" + "=" * 80)
print("📊 SYNTHETIC DATA GENERATION SUMMARY")
print("=" * 80)
print(f"Target Table:        {TARGET_TABLE}")
print(f"Generator Version:   v2.0 (Phase 1 Compatible)")
print(f"\n📈 COUNTS:")
print(f"   Initial Rows:     {initial_row_count:,}")
print(f"   Generated Rows:   {ROWS_TO_GENERATE:,}")
print(f"   Final Row Count:  {final_count:,}")
print(f"   Verification:     {'✅ PASSED' if final_count == expected_count else '❌ FAILED'}")
print(f"\n⚡ PERFORMANCE:")
print(f"   Generation Time:  {generation_time:.2f}s")
print(f"   Write Time:       {write_time:.2f}s")
print(f"   Total Time:       {total_time:.2f}s")
print(f"   Throughput:       {throughput:,.0f} rows/sec")
print(f"\n🔧 AUDIT COLUMNS:")
if ENABLE_AUDIT_COLUMNS and all(audit_columns_present.values()):
    print(f"   Status:           ✅ Populated with timestamps")
    print(f"   created_timestamp: Current timestamp")
    print(f"   updated_timestamp: Current timestamp")
    print(f"   is_deleted:       False")
else:
    print(f"   Status:           ⚠️  Set to NULL (pre-Phase 1 compatibility)")
print("=" * 80)

# COMMAND ----------

# MAGIC %md
# MAGIC ## USAGE NOTES
# MAGIC 
# MAGIC ### **Testing Incremental Load:**
# MAGIC 
# MAGIC ```python
# MAGIC # Run 1: Generate 1M records
# MAGIC ROWS_TO_GENERATE = 1000000
# MAGIC ENABLE_AUDIT_COLUMNS = True
# MAGIC # All records get created_timestamp = NOW
# MAGIC 
# MAGIC # Wait 5 minutes
# MAGIC 
# MAGIC # Run 2: Generate another 100K records
# MAGIC ROWS_TO_GENERATE = 100000
# MAGIC ENABLE_AUDIT_COLUMNS = True
# MAGIC # New records get created_timestamp = NOW + 5 minutes
# MAGIC 
# MAGIC # Now your ETL can detect:
# MAGIC # SELECT * FROM person WHERE created_timestamp > last_watermark
# MAGIC # Will return only the 100K new records ✅
# MAGIC ```
# MAGIC 
# MAGIC ### **Backward Compatibility:**
# MAGIC 
# MAGIC ```python
# MAGIC # If running on pre-Phase 1 table (no audit columns):
# MAGIC ENABLE_AUDIT_COLUMNS = False
# MAGIC # Generator will work, audit columns set to NULL
# MAGIC ```
# MAGIC 
# MAGIC ### **Performance Tuning:**
# MAGIC 
# MAGIC ```python
# MAGIC # For < 1M rows:
# MAGIC ROWS_TO_GENERATE = 100000  # Fast (~1 second)
# MAGIC 
# MAGIC # For 1-10M rows:
# MAGIC ROWS_TO_GENERATE = 5000000  # Medium (~15 seconds)
# MAGIC 
# MAGIC # For 10M+ rows:
# MAGIC # Run multiple batches instead of one large batch
# MAGIC ```

# COMMAND ----------

# MAGIC %md
# MAGIC ---
# MAGIC ## ✅ PHASE 1 - STEP 2 COMPLETE
# MAGIC 
# MAGIC **What Was Done:**
# MAGIC - ✅ Updated generator to support audit columns
# MAGIC - ✅ New records get proper timestamps (not NULL)
# MAGIC - ✅ Maintains backward compatibility
# MAGIC - ✅ Quality checks added
# MAGIC - ✅ Post-insert verification
# MAGIC 
# MAGIC **What's Next:**
# MAGIC - ⏭️ Step 3: Create ETL control/watermark table
# MAGIC - ⏭️ Step 4: Implement incremental load in ETL v4.0
# MAGIC 
# MAGIC **Testing Incremental Load:**
# MAGIC 1. Run this generator to create new records (they get timestamps)
# MAGIC 2. Run ETL v4.0 (Step 4) which will only process new records
# MAGIC 3. Verify 99% runtime reduction ✅


StatementMeta(, efbdbbf8-4157-46bb-9b38-67c3a1256b91, 3, Finished, Available, Finished, False)

SYNTHETIC DATA GENERATOR v2.0 - PHASE 1 COMPATIBLE
Spark Version: 3.5.5.5.4.20260109.1
Execution Time: 2026-03-01 07:23:49
📋 CONFIGURATION:
   Target Table: Lake24.dbo.person
   Rows to Generate: 1,000,000
   Audit Columns: Enabled ✅

🎬 Starting generation of 1,000,000 records...

🔍 TABLE ANALYSIS:
--------------------------------------------------------------------------------
   Existing Records: 15,712,818
   Current Max ID:   312,960,608
   Next ID:          312,960,609

📊 SCHEMA ANALYSIS:
   Total Columns:      21
   created_timestamp:  ✅ Present
   updated_timestamp:  ✅ Present
   is_deleted:         ✅ Present
   Status: ✅ Phase 1 schema detected (audit columns present)
--------------------------------------------------------------------------------

⚡ GENERATING 1,000,000 SYNTHETIC RECORDS:
--------------------------------------------------------------------------------
   ✅ Generated base 18 columns
   ✅ Added audit columns with timestamps
      - created_timestamp: 2026-03-01 